### Terrain & hydro analysis

The following workflow demonstrates a typical set of land surface parameters used in many environmental applications, particulalrly geomorphology, soil science and hydrology. These are just a few brief examples as there are so many elevation derivatives out there it is best to explore further dependent on the application.

Likely the best resource on this is (albeit a bit old now.....):

Hengl, T., & Reuter, H. I. (2009). Geomorphometry: Concepts, Software, Applications. (T. Hengl & H. I. Reuter, Eds.) (Vol. 33). Amsterdam: Elsevier.

Elsewhere - this website 
http://www.geomorphometry.org/

I personally used similar variables in paper during my PhD for semi-automated mapping of terrain etc.
Robb, C., Willis, I., Arnold, N., & Guðmundsson, S. (2015). A semi-automated method for mapping glacial geomorphology tested at Breiðamerkurjökull, Iceland. Remote Sensing of Environment. 

Ultimately, LSPs are usually some of the imput layers used by goverment agencies to site the regeneration of woodland, habitats etc. as well as model the outcome of government policies given changing land use. CEH and Welsh Gov's ERAMMP (https://erammp.wales/en) system and the internationally used LUCI modelhttps://www.lucitools.org/ are good examples of this. 


The software lib used here (though several others do the same things) is whitebox tools, written in rust with a python API.

Install:

```conda install -c conda-forge whitebox_tools ```

A LiDAR-derived DTM has been obtained from Wels-gov's repo for this purpose.

The data is as ever available here:

https://drive.google.com/file/d/17_TqgVnBHWltwrMkf5qkyIWv_yqD-X3x/view?usp=sharing

Save it an appropriate place then change to this directory. 

Please use QGIS to visualise these parameters as this is far more versitile than matplotlib. 
Viewed as a hillshade, the DTM should look like this. 

<img src="Figures/dem.png" style="height:500px">

In [ ]:
from WBT.whitebox_tools import WhiteboxTools as wbt

import os

os.chdir("insert working dir here")

wbt = WhiteboxTools()

wrkDir = os.getcwd()

inRas = "dem.tif"

### Hydrology

We will go through typical flow accumulation workflow and extract a stream network.

First, we fill any holes in the DEM surface itself

In [ ]:
wbt.fill_missing_data(i=inRas, output="filled.tif", filter=11)

Next, we breach any depressions in the DEM that will result in artefacts in the flow accumulation raster.
The differences may seem fairly imperceptable from the original DTM, but will have a positive effect on the end result.  

In [ ]:
wbt.breach_depressions(em="filled.tif", output="enforced.tif")


Finally, we produce a flow accumulation raster, which, as the name suggests, models flow accross the surface. This uses the D-Infinity algorithm of Tarbuton (1997), which attempts to model partial divergence in flow.

The ```log=True``` parameter makes this easier to visualise in GIS.  

In [ ]:
wbt.d_inf_flow_accumulation(dem="enforced.tif", output="Dinf_flow.tif", log=True)

#write an additional non-log for further analysis

wbt.d_inf_flow_accumulation(dem="enforced.tif", output="Dinf_flow_final.tif", log=False)


The results, using the log enhanced file will look something like this with judicious use of symbology.

<img src="Figures/dinf.png" style="height:500px">

Clearly this, as with all similar variants of flow accumulation does not perform so well in wider areas of modest topographic variability. Ways round this include simply smoothing the input DTM or use a coarser data source (SRTM or something - still smoothing this) having followed the same procedure. 

### Stream Network

Having seen the breakdown, we will now run the stages in one, outputting a D8 flow accumulation raster.

D-8* is a less complex (and pretty old) algorithm, but is still likely best for attemtpting to extract a stream network,to provide an approximation of the main lines of drainage. 

*D-8 only maps convergent flow over one pixel



In [ ]:

wbt.flow_accumulation_full_workflow(
    inRas, 
    "smoothdem.tif", 
    "pntr.tif", 
    "flowaccum.tif", 
    out_type="Specific Contributing Area", 
    log=False, 
    clip=False, 
    esri_pntr=False, 
)

Now these two commands will extract a polyline shapefile of the network.

The critical parameter is the threshold (```7000```), which is a guess at the flow accumulation score that results in drainage as a stream network as interpreted by the user. This is not ideal, but a robust empirical solution is not available.  

In [ ]:
wbt.extract_streams( "flowaccum.tif", "streams.tif", 7000, zero_background=False)

wbt.raster_streams_to_vector( "streams.tif",  "pntr.tif", "streams.shp", esri_pntr=False)

The network looks fairly sensible, but experimentation is required to get a better approximation. 

<img src="Figures/streams.png" style="height:500px">


The distance above stream will give us a mini-catchment distance from which you could get a quick and dirty flood extent, though this is a GROSS SIMPLIFICATION. 

You would usually use a 2D hydraulic model to do this properly (e.g. lisflood, CCH_HE), but as a rough idea it is ok.

In [ ]:
wbt.elevation_above_stream("smoothdem.tif",  "streams.tif", "abovestrm.tif")


This will look something like...

<img src="Figures/elstr.png" style="height:500px">

Where darker blue is closer to the stream network.

By thresholding this we can define an approximate flood plain for each polyline to a defined height.

#### Wetness

A related LSP is the topographic wetness index.

The index describes the propensity for a site to be saturated to the surface given its contributing area and local slope characteristics.

This requires the DEM gradient to be calculated first as an input variable. 

In [ ]:
wbt.slope("enforced.tif", "slope.tif")

wbt.wetness_index("flow_accum.tif", "slope.tif", "twi.tif")

With similar visualisation settings the twi looks as follows:

<img src="Figures/twi.png" style="height:500px">

It is notable that both these hydro-LSPs suffer from artifacts caused by the filtering process applied to the LiDAR point cloud and resulting interpolation over sparse ground returns. 

e.g.

<img src="Figures/junk.png" style="height:200px">


### Relative height

Representing relative height (peaks and troughs) is another useful variable. 
The elevation percentile filter derives a good representation of relative elevation, normalised to 0 > 100. By increasing the window size, depressions and ridges of increasing size are accentuated.

Every pixel is 5x5m so a window of 5 equates to 25m.



In [ ]:
wbt.elev_percentile("enforced.tif", "elper5.tif", filterx=5, filtery=5, sig_digits=2)

wbt.elev_percentile("enforced.tif", "elper5.tif", filterx=20, filtery=20, sig_digits=2)

Thus window 5:

<img src="Figures/elper5.png" style="height:500px">

window 20:

<img src="Figures/elper20.png" style="height:500px">